In [29]:
# -*- coding: utf-8 -*-
from Tkinter import *
import PIL
from PIL import Image, ImageTk, ImageDraw

from tkFileDialog import askopenfilename

#import potrebnih biblioteka
import cv2
import collections
import numpy as np
import scipy as sc
import matplotlib.pyplot as plt
from scipy.spatial import distance

# k-means
from sklearn.cluster import KMeans

# keras
from keras.models import Sequential
from keras.layers.core import Dense,Activation
from keras.optimizers import SGD


import matplotlib.pylab as pylab
import re
pylab.rcParams['figure.figsize'] = 16, 12 # za prikaz većih slika i plotova, zakomentarisati ako nije potrebno
#Funkcionalnost implementirana u V1
def load_image(path):
    return cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)
def image_gray(image):
    return cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
def image_bin(image_gs):
    ret,image_bin = cv2.threshold(image_gs, 127, 255, cv2.THRESH_BINARY)
    return image_bin
def image_bin_adaptive(image_gs):
    image_bin = cv2.adaptiveThreshold(image_gs, 255, cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY,35,10)
    return image_bin
def invert(image):
    return 255-image
def display_image(image, color= False):
    if color:
        plt.imshow(image)
    else:
        plt.imshow(image, 'gray')
def dilate(image):
    kernel = np.ones((3,3)) # strukturni element 3x3 blok
    return cv2.dilate(image, kernel, iterations=1)
def erode(image):
    kernel = np.ones((3,3)) # strukturni element 3x3 blok
    return cv2.erode(image, kernel, iterations=1)
def erode2(image):
    kernel = np.ones((2,2)) # strukturni element 2x2 blok
    return cv2.erode(image, kernel, iterations=1)

#Funkcionalnost implementirana u V2
def resize_region(region):
    resized = cv2.resize(region,(28,28), interpolation = cv2.INTER_NEAREST)
    return resized
def scale_to_range(image):
    return image / 255
def matrix_to_vector(image):
    return image.flatten()
def prepare_for_ann(regions):
    ready_for_ann = []
    for region in regions:
        ready_for_ann.append(matrix_to_vector(scale_to_range(region)))
    return ready_for_ann
def convert_output(outputs):
    return np.eye(len(outputs))
def winner(output):
    return max(enumerate(output), key=lambda x: x[1])[0]
# Uklanjanje šuma
def remove_noise(binary_image):
    ret_val = erode2(dilate(binary_image))
    ret_val = invert(ret_val)
    return ret_val

def select_roi(image_orig, image_bin):
    '''
    Funkcija kao u vežbi 2, iscrtava pravougaonike na originalnoj slici, pronalazi sortiran niz regiona sa slike,
    i dodatno treba da sačuva rastojanja između susednih regiona.
    '''
    img, contours, hierarchy = cv2.findContours(image_bin.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    #Način određivanja kontura je promenjen na spoljašnje konture: cv2.RETR_EXTERNAL
    regions_dict = {}
    for contour in contours: 
        x,y,w,h = cv2.boundingRect(contour)
        area = cv2.contourArea(contour)
        #if area > 10000 and area < 15000 and h < 100 and h > 12 and w > 50:
        #if area > 8000 and h > 50 and h < 70 and w > 250 and w < 290:
        if area > 8000 and h > 40 and h < 105 and w > 180 and w < 350:
            print area
            
            region = image_bin[y:y+h+1,x:x+w+1];
            # Proširiti regions_dict elemente sa vrednostima boundingRect-a ili samim konturama
            regions_dict[x] = [resize_region(region), (x,y,w,h)]
            cropp = (x, y, x+w, y+h)
            cv2.rectangle(image_orig,(x,y),(x+w,y+h),(0,255,0),2)
            
            
    sorted_regions_dict = collections.OrderedDict(sorted(regions_dict.items()))
    sorted_regions = np.array(sorted_regions_dict.values())
    
    sorted_rectangles = sorted_regions[:,1]
    region_distances = [-sorted_rectangles[0][0]-sorted_rectangles[0][2]]
    # Izdvojiti sortirane parametre opisujućih pravougaonika
    # Izračunati rastojanja između svih susednih regiona po x osi i dodati ih u region_distances niz
    for x,y,w,h in sorted_regions[1:-1, 1]:
        region_distances[-1] += x
        region_distances.append(-x-w)
    region_distances[-1] += sorted_rectangles[-1][0]
    
    print 'SELETKOVAO TABLICU i IZLAZI'
    return image_orig, sorted_regions[:, 0], region_distances, cropp
    


def tablica():
    print 'tablica()'
    image_color = load_image('Images/resized2.jpg')
    img2 = image_bin(image_gray(image_color))
    img2_erode = erode((img2))
    selected_regions2, letters2, distances2, cropp = select_roi(image_color.copy(), img2_erode)
    #display_image(selected_regions2)
    print 'izlazak iz tablica()'
    return cropp

def set_Pic_path(text):
    print 'set_Pic_path text: '
    print text
    lblIspis("       ")
    lblLicencePlace("")
    img = Image.open(text)
    img = img.resize((800, 600), PIL.Image.ANTIALIAS)
    img_prikaz = img.resize((700, 440), PIL.Image.ANTIALIAS)

    img.save('Images/resized.jpg')

    photo = ImageTk.PhotoImage(img_prikaz)
    labelSlika = Label(image=photo)
    labelSlika.image = photo
    labelSlika.place(x=120, y=20)
    
    img2 = Image.open(text)
    img2 = img2.resize((900, 600), PIL.Image.ANTIALIAS)
    dr = ImageDraw.Draw(img2)

    
    img2.save('Images/resized2.jpg')
    dr.rectangle(tablica(), outline = "green")
    img2.save('Images/resized2.jpg')   
    
    
    img_resize_crop = Image.open('Images/resized2.jpg')
    img_resize_crop.crop(tablica()).save('Images/tablica_cropovano.jpg')
    print 'ZAVRSIO KROPOVANJE'
#     lblLicencePlace('tablica_cropovano.jpg')
    testLicencePlate()
    return
def testLicencePlate():
    print 'POKUSAJ LOADOVANJA SLIKE'
    image_test_original_obucavanje = load_image('Images/tablica_cropovano.jpg')
    
    display_image(image_test_original_obucavanje)
    image_test_obucavanje = invert(erode(dilate(image_bin_adaptive(image_bin(image_gray(image_test_original_obucavanje))))))
    selected_test_obucavanje, letters_obucavanje = select_roi_for_neuron(image_test_original_obucavanje.copy(),image_test_obucavanje)
    
    img = Image.fromarray(selected_test_obucavanje, 'RGB')
    img.save('Images/tablice_regioni.jpg')
    lblLicencePlace('Images/tablice_regioni.jpg')
    
    print 'Broj prepoznatih regiona:'
    if len(letters_obucavanje) == 0 :
        print 'NIJE NISTA SELEKTOVAO'
        return
    print len(letters_obucavanje)
    inputs_test = prepare_for_ann(letters_obucavanje)
    results_test = ann.predict(np.array(inputs_test, np.float32))

    ispis = display_result(results_test, alphabet1)

    print display_result(results_test, alphabet1)

    display_image(selected_test_obucavanje)
    
    
    lblIspis(ispis)
    
    return 
def lblLicencePlace(text):
    if text != "":
        img = Image.open(text)
        img = img.resize((360, 80), PIL.Image.ANTIALIAS)

#         img.save('resized.jpg')

        photo = ImageTk.PhotoImage(img)
        labelSlika = Label( image=photo)
        labelSlika.image = photo
        labelSlika.place(x=260, y=500)

        return
    else:
        img = Image.open('Images/tablica_prazno.jpg')
        img = img.resize((360, 80), PIL.Image.ANTIALIAS)
        photo = ImageTk.PhotoImage(img)
        labelSlika = Label( image=photo)
        labelSlika.image = photo
        labelSlika.place(x=260, y=500)
def lblIspis(tablica):

    tmpIspis = StringVar()
    #print "TABLICA"
    #print tablica
    #if(tablica[2]=='U' and tablica[3]=='0'):
    #    tablica = re.sub('[U]', '', tablica)
    # print 'TMPISPIS:'
    # print tmpIspis
    var = StringVar()
    labelIspis = Label( font=("Helvetica", 14), text="Car licence plate:")
#     labelIspis.pack()
    labelIspis.place(x=660, y=490)

    var = StringVar()
    labelIspis = Label( font=("Helvetica", 14), text=tablica,borderwidth=2,
             background='white',
             relief='solid')
    labelIspis.pack()
    labelIspis.place(x=660, y=530)    
def load_image( path):
    return cv2.cvtColor(cv2.imread(path), cv2.COLOR_BGR2RGB)

def openFile():
    Tk().withdraw()  # we don't want a full GUI, so keep the root window from appearing
    filename = askopenfilename(initialdir="/home/student/ZADACI/Licence plate recognition project/Images/Miki", title="choose your file",
                                               filetypes=(("jpeg files", "*.jpg"), ("all files", "*.*")))
    #location = askopenfilename()  # show an "Open" dialog box and return the path to the selected file
    #  radiSve(location)
    #dugmeSacuvaj = Button(root, text="Write data in file", width=20, height=5, command=lambda: run(ann, alphabet))
    #dugmeSacuvaj.place(x=530, y=500)
    print 'openFile: '
    print(filename)
    return filename

def initialize():
    root = Tk()
    root.resizable(0, 0)
    root.title("Licence plate recognition software")
    root.geometry('1000x650+20+20')

    ontBold = ('Arial', 12, 'bold')
    fontSerial = ('Arial', 12)
    fontTextField = ('Arial', 16)

    root.dugmeUcitaj = Button( text="Chose image", width=20, height=5,
                         command=lambda: set_Pic_path(openFile()), font=("Helvetica", 13))
    root.dugmeUcitaj.place(x=30, y=500)
    return root



# TODO 1 - Rotiranje regiona
def rotate_regions(contours, angles, centers, sizes):
    '''Funkcija koja vrši rotiranje regiona oko njihovih centralnih tačaka
    Args:
        contours: skup svih kontura [kontura1, kontura2, ..., konturaN]
        angles:   skup svih uglova nagiba kontura [nagib1, nagib2, ..., nagibN]
        centers:  skup svih centara minimalnih pravougaonika koji su opisani
                  oko kontura [centar1, centar2, ..., centarN]
        sizes:    skup parova (height,width) koji predstavljaju duzine stranica minimalnog
                  pravougaonika koji je opisan oko konture [(h1,w1), (h2,w2), ...,(hN,wN)]
    Return:
        ret_val: rotirane konture'''
    ret_val = []
    for idx, contour in enumerate(contours):

        angle = angles[idx]
        cx, cy = centers[idx]
        height, width = sizes[idx]
        if width < height:
            angle += 90

        # Rotiranje svake tačke regiona oko centra rotacije
        alpha = np.pi / 2 - abs(np.radians(angle))
        region_points_rotated = np.ndarray((len(contour), 2), dtype=np.int16)
        for i, point in enumerate(contour):
            x = point[0]
            y = point[1]

            # TODO 1 - izračunati koordinate tačke nakon rotacije
            rx = np.sin(alpha) * (x - cx) - np.cos(alpha) * (y - cy) + cx
            ry = np.cos(alpha) * (x - cx) + np.sin(alpha) * (y - cy) + cy

            region_points_rotated[i] = [rx, ry]
        ret_val.append(region_points_rotated)

    return ret_val

def merge_regions(contours):
    '''Funkcija koja vrši spajanje kukica i kvačica sa osnovnim karakterima
    Args:
        contours: skup svih kontura (kontura - niz tacaka bele boje)
    Return:
        ret_val: skup kontura sa spojenim kukicama i kvacicama'''
    ret_val = []
    merged_index = [] #lista indeksa kontura koje su već spojene sa nekim
    indexes = []
    for i,contour1 in enumerate(contours):
        
        min_y = min(contour1[:,1])
        max_y = max(contour1[:,1])
        
        for j,contour2 in enumerate(contours): #kukice
            
            min_y2 = min(contour2[:,1])
            max_y2 = max(contour2[:,1])
            if((-(max_y - min_y - max_y2+min_y2) < 0) and (-(max_y - min_y - max_y2+min_y2) > -30)):
                indexes.append(j)
                print'indeks:'
                print j
        break
    for i,contour1 in enumerate(contours): #slova
        if i in merged_index:
            continue
        min_x1 = min(contour1[:,0])
        max_x1 = max(contour1[:,0])
        min_y1 = min(contour1[:,1])
        max_y1 = max(contour1[:,1])
        for j,contour2 in enumerate(contours): #kukice
            if j in merged_index or i == j:
                continue
            min_x2 = min(contour2[:,0])
            max_x2 = max(contour2[:,0])
            min_y2 = min(contour2[:,1])
            max_y2 = max(contour2[:,1])
            
            #TODO 2 - izvršiti spajanje kukica iznad slova
            #spajanje dva niza je moguće obaviti funkcijom np.concatenate((contour1,contour2))
            
#             if len(contour1)*0.5>len(contour2): #provera pretpostavke da je contour1 slovo
                
#                 if (min_y1-max_y2)<max(max_y1-min_y1,max_y2-min_y2)/2 \
#                 and (min_x2>min_x1-5 and max_x2<max_x1+5):
#                     #spajanje kontura
#                     ret_val.append(np.concatenate((contour1,contour2)))
#                     merged_index.append(i)
#                     merged_index.append(j)
            foundI = False
            foundJ = False
            for k in indexes:
                if(k==i):
                    foundI = True
                if(k == j):
                    foundJ = True
            if(foundI and foundJ):
                merged_index.append(i)
                merged_index.append(j)
    #svi regioni koji se nisu ni sa kim spojili idu u listu kontura, bez spajanja
    for idx,contour in enumerate(contours):
        if idx not in merged_index:
            ret_val.append(contour)
        
    return ret_val

def select_roi_proba(image_orig, image_bin):
    
    img, contours_borders, hierarchy = cv2.findContours(image_bin.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    
    contours = []
    contour_angles = []
    contour_centers = []
    contour_sizes = []
    for contour in contours_borders:
        center, size, angle = cv2.minAreaRect(contour)
        xt,yt,h,w = cv2.boundingRect(contour)

        region_points = []
        for i in range (xt,xt+h):
            for j in range(yt,yt+w):
                dist = cv2.pointPolygonTest(contour,(i,j),False)
                if dist>=0 and image_bin[j,i]==255: # da li se tacka nalazi unutar konture?
                    region_points.append([i,j])
        contour_centers.append(center)
        contour_angles.append(angle)
        contour_sizes.append(size)
        contours.append(region_points)
    
    #Postavljanje kontura u vertikalan polozaj
    contours = rotate_regions(contours, contour_angles, contour_centers, contour_sizes)
    
    #spajanje kukica i kvacica
    contours = merge_regions(contours)
    
    regions_dict = {}
    for contour in contours:
    
        min_x = min(contour[:,0])
        max_x = max(contour[:,0])
        min_y = min(contour[:,1])
        max_y = max(contour[:,1])

        region = np.zeros((max_y-min_y+1,max_x-min_x+1), dtype=np.int16)
        for point in contour:
            x = point[0]
            y = point[1]
            
             # TODO 3 - koordinate tacaka regiona prebaciti u relativne koordinate
            '''Pretpostavimo da gornja leva tačka regiona ima apsolutne koordinate (100,100).
            Ako uzmemo tačku sa koordinatama unutar regiona, recimo (105,105), nakon
            prebacivanja u relativne koordinate tačka bi trebala imati koorinate (5,5) unutar
            samog regiona.
            '''
            region[y-min_y,x-min_x] = 255

        
        regions_dict[min_x] = [resize_region(region), (min_x,min_y,max_x-min_x,max_y-min_y)]
        
    sorted_regions_dict = collections.OrderedDict(sorted(regions_dict.items()))
    sorted_regions = np.array(sorted_regions_dict.values())
    
    sorted_rectangles = sorted_regions[:,1]
    region_distances = [-sorted_rectangles[0][0]-sorted_rectangles[0][2]]
    for x,y,w,h in sorted_regions[1:-1, 1]:
        region_distances[-1] += x
        region_distances.append(-x-w)
    for x,y,w,h in sorted_regions[0:-1, 1]:
        cv2.rectangle(image_orig,(x,y),(x+w,y+h),(0,255,0),2)
    region_distances[-1] += sorted_rectangles[-1][0]
    
    return image_orig, sorted_regions[:, 0], region_distances


def select_roi_for_neuron_proba(image_orig, image_bin):
    '''Oznaciti regione od interesa na originalnoj slici. (ROI = regions of interest)
        Za svaki region napraviti posebnu sliku dimenzija 28 x 28.
        Za označavanje regiona koristiti metodu cv2.boundingRect(contour).
        Kao povratnu vrednost vratiti originalnu sliku na kojoj su obeleženi regioni
        i niz slika koje predstavljaju regione sortirane po rastućoj vrednosti x ose
    '''
    img, contours, hierarchy = cv2.findContours(image_bin.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    sorted_regions = []  # lista sortiranih regiona po x osi (sa leva na desno)
    regions_dic = {}
    x0, y0, w0, h0 = cv2.boundingRect(contours[0])  # koordinate i velicina granicnog pravougaonika

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)  # koordinate i velicina granicnog pravougaonika
        area = cv2.contourArea(contour)
        # CIFRE, x, y, +

        # kopirati [y:y+h+1, x:x+w+1] sa binarne slike i smestiti u novu sliku
        # označiti region pravougaonikom na originalnoj slici (image_orig) sa rectangle funkcijom
        region = image_bin[y:y + h + 1, x:x + w + 1];
        regions_dic[x] = resize_region(region)
        cv2.rectangle(image_orig, (x, y), (x + w, y + h), (0, 255, 0), 2)
    # # minus


    sorted_regions_dic = collections.OrderedDict(sorted(regions_dic.items()))
    sorted_regions = sorted_regions_dic.values()

    # sortirati sve regione po x osi (sa leva na desno) i smestiti u promenljivu sorted_regions
    return image_orig, sorted_regions


def create_ann1():
    ann = Sequential()
    # Postavljanje slojeva neurona mreže 'ann'
    ann.add(Dense(input_dim=784, output_dim=256, init="glorot_uniform"))
    ann.add(Activation("sigmoid"))
    ann.add(Dense(input_dim=256, output_dim=40, init="glorot_uniform"))
    ann.add(Activation("sigmoid"))
    return ann


def train_ann(ann, X_train, y_train):
    X_train = np.array(X_train, np.float32)
    y_train = np.array(y_train, np.float32)

    # definisanje parametra algoritma za obucavanje
    sgd = SGD(lr=0.01, momentum=0.9)
    ann.compile(loss='mean_squared_error', optimizer=sgd)

    # obucavanje neuronske mreze
    ann.fit(X_train, y_train, nb_epoch=3000, batch_size=1, verbose=0, shuffle=False, show_accuracy=False)

    return ann




def display_result(outputs, alphabet):
    '''
    Funkcija određuje koja od grupa predstavlja razmak između reči, a koja između slova, i na osnovu
    toga formira string od elemenata pronađenih sa slike.
    Args:
        outputs: niz izlaza iz neuronske mreže.
        alphabet: niz karaktera koje je potrebno prepoznati
        kmeans: obučen kmeans objekat
    Return:
        Vraća formatiran string
    '''
    # Odrediti indeks grupe koja odgovara rastojanju između reči, pomoću vrednosti iz k_means.cluster_centers_
    # w_space_group = max(enumerate(k_means.cluster_centers_), key = lambda x: x[1])[0]
    result = alphabet[winner(outputs[0])]
    for idx, output in enumerate(outputs[1:, :]):
        # Iterativno dodavati prepoznate elemente kao u vežbi 2, alphabet[winner(output)]
        # Dodati space karakter u slučaju da odgovarajuće rastojanje između dva slova odgovara razmaku između reči.
        # U ovu svrhu, koristiti atribut niz k_means.labels_ koji sadrži sortirana rastojanja između susednih slova.
        # if (k_means.labels_[idx] == w_space_group):
        # result += ' '
        result += alphabet[winner(output)]
    return result

def select_roi_for_neuron(image_orig, image_bin):
    '''Oznaciti regione od interesa na originalnoj slici. (ROI = regions of interest)
        Za svaki region napraviti posebnu sliku dimenzija 28 x 28.
        Za označavanje regiona koristiti metodu cv2.boundingRect(contour).
        Kao povratnu vrednost vratiti originalnu sliku na kojoj su obeleženi regioni
        i niz slika koje predstavljaju regione sortirane po rastućoj vrednosti x ose
    '''
    img, contours, hierarchy = cv2.findContours(image_bin.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    sorted_regions = []  # lista sortiranih regiona po x osi (sa leva na desno)
    regions_dic = {}
    x0, y0, w0, h0 = cv2.boundingRect(contours[0])  # koordinate i velicina granicnog pravougaonika

    for contour in contours:
        x, y, w, h = cv2.boundingRect(contour)  # koordinate i velicina granicnog pravougaonika
        area = cv2.contourArea(contour)
        # CIFRE, x, y, +
        if h > 21 and w > 4 and h < 102 and w < 70 :
            # kopirati [y:y+h+1, x:x+w+1] sa binarne slike i smestiti u novu sliku
            # označiti region pravougaonikom na originalnoj slici (image_orig) sa rectangle funkcijom
            region = image_bin[y:y + h + 1, x:x + w + 1];
            regions_dic[x] = resize_region(region)
            cv2.rectangle(image_orig, (x, y), (x + w, y + h), (0, 255, 0), 2)
            #         # minus

    sorted_regions_dic = collections.OrderedDict(sorted(regions_dic.items()))
    sorted_regions = sorted_regions_dic.values()

    # sortirati sve regione po x osi (sa leva na desno) i smestiti u promenljivu sorted_regions
    return image_orig, sorted_regions


In [3]:
image_test_original_obucavanje = load_image('Images/dataset/obucavanje.jpg')
# display_image(image_test_original_obucavanje)
image_test_obucavanje = invert(image_bin(image_gray(image_test_original_obucavanje)))

selected_test_obucavanje, letters_obucavanje = select_roi_for_neuron_proba(image_test_original_obucavanje.copy(),
                                                                           image_test_obucavanje)
print 'Broj prepoznatih regiona:'
print len(letters_obucavanje)
display_image(selected_test_obucavanje)

alphabet1 = ['A', 'B', 'C', 'Ć', 'Č', 'D', 'Đ', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'R',
             'S',
             'Š', 'T', 'U', 'V', 'Z', 'Ž', 'W', 'X', 'Y', '1', '2', '3', '4', '5', '6', '7', '8', '9', '0']

inputs = prepare_for_ann(letters_obucavanje)
outputs = convert_output(alphabet1)
print len(inputs)
print len(outputs)
    
ann = create_ann1()
ann = train_ann(ann, inputs, outputs)


In [30]:
#inputs_test = app.prepare_for_ann(letters_obucavanje)
#results_test = ann.predict(np.array(inputs_test, np.float32))

#ispis = app.display_result(results_test, alphabet1)

#print app.display_result(results_test, alphabet1)

def quit():
    # check if saving
    # if not:
    root.destroy()

root = initialize()
root.protocol('WM_DELETE_WINDOW', quit)  # root is your root window

root.mainloop()


KeyboardInterrupt: 